In [1]:
"""
CancerDiscovery
Figure 1
OLINK Analysis

This notebook follows the following analysis format
- Processing
- Statistical tests
- Barplot (Sinai)
- Barplot (validation cohort)
"""

__author__ = "Daniel Ranti"
__credits__ = "Yuan-Shuo Wang"
__license__ = "Open Access"
__version__ = "1.0.1"
__maintainer__ = "Daniel Ranti"
__email__ = "daniel.l.ranti@gmail.com"
__status__ = "Development"

In [ ]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

urine_list = ['AHBCG03 urine',
'AHBCG03_3 Urine',
'AHBCG03_04 urine',
'AHBCG03_4 urine',
'AHBCG08_1 urine',
'AHBCG09_1 urine',
'AHBCG07_5 urine',
'AHBCG11_1 urine',
'AHBCG10_1 urine',
'AHBCG09_3 urine',
'AHBCG10_4 urine',
'AHBCG08_4 urine',
'AHBCG12_1 urine',
'AHBCG12_2 urine',
'24920_3 urine',
'AHBCG10_3 urine',
'AHBCG12_3 urine',
'24920_4 urine',
'AHBCG13_1 urine',
'8408608_9 urine',
'AHBCG12_4 urine',
'AHBCG13_3 urine',
'8408608_7 urine',
'5660914_urine',
'8408608#1_urine',
'8215908_urine',
'2305870_2 urine',
'E539104_urine',
'2258786_urine',
'8408608#2_urine',
'8408608#3_urine',
'A100230 urine',
'AHBCG01#2 urine',
'5044122 urine',
'AHBCG04 urine',
'AHBCG05 urine',
'AHBCG03_2 Urine',
'E579150 urine',
'E579150_2 urine',
'E579150_3 urine',
'E579150_4 urine',
'A194053 urine',
'AHBCG0103 urine',
'E606918 urine',
'AHBCG04_02 urine',
'AHBCG05_02 urine',
'AHBCG06 urine',
'7575386 urine',
'E360058 urine',
'E663619 urine',
'AHBCG06_2 urine',
'AHBCG04_3 urine',
'AHBCG04_4 urine',
'AHBCG05_3 urine',
'8408608_4 urine',
'AHBCG07_1 urine',
'AHBCG07_2 urine',
'AHBCG06_3 urine',
'2540961 urine',
'A194053_2 urine',
'2305870_4 urine',
'8408608_5 urine',
'7520157_urine',
'E686525_urine',
'AHBCG07_3 urine',
'AHBCG05_4 urine',
'8408608_6 urine',
'AHBCG13_1 urine',
'2305870_3 urine',]

urine_list = [str(i) for i in urine_list]

#### Functions to help create grouped labels
from itertools import groupby
def add_line(ax, xpos, ypos):
    line = plt.Line2D([xpos, xpos], [ypos + .1, ypos],
                      transform=ax.transAxes, color='black')
    line.set_clip_on(False)
    ax.add_line(line)

def label_len(my_index,level):
    labels = my_index.get_level_values(level)
    return [(k, sum(1 for i in g)) for k,g in groupby(labels)]

def label_group_bar_table(ax, df):
    ax.axvline(x=0, color='k',linewidth=5)
    ypos = -0.02
    scale = 1./df.index.size
    for level in range(df.index.nlevels)[::-1]:
        pos = 0
        for label, rpos in label_len(df.index,level):
            print(label, rpos)
            lxpos = (pos + .5 * rpos)*scale
            ax.text(lxpos, ypos, label, rotation=90, ha='center', va='top',transform=ax.transAxes)
            ax.axvline(x=pos,ymin=0, color='k',linewidth=6, clip_on=False)
            pos += rpos
        ypos -= .1
    ax.axvline(x=pos,ymin=0, color='k',linewidth=4, clip_on=False)

class MidPointLogNorm(LogNorm):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        LogNorm.__init__(self,vmin=vmin, vmax=vmax, clip=clip)
        self.midpoint=midpoint
    def __call__(self, value, clip=None):
        x, y = [np.log(self.vmin), np.log(self.midpoint), np.log(self.vmax)], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(np.log(value), x, y))

# Gotta do it to em
import warnings
warnings.filterwarnings('ignore')  

In [ ]:
######################################### 
# Importing and generating URINE NPX Data 
######################################### 

print('Importing Normalized Data')
raw_data = pd.read_excel('Olink data.xlsx', sheet_name='post bt-lot norm')
raw_data = raw_data.set_index('Unnamed: 0').T

print('Import HC data')
hc_urine = pd.read_excel('olink_plt5_normed.xlsx').replace({'Unnamed: 0':'cytokine'})

# Importing Jorge's Biorepository spreadsheet
biorepository_info = pd.read_excel('new biorepository db.xlsx')

# Looking for sample ID matches between the biorepository database and the 
matches = []
bio_ids = []
for bio_id in biorepository_info.Patient:  
    result = list(filter(lambda x: x.startswith(str(bio_id)[0:5]), list(raw_data.index)))
    if len(result) > 0:
        matches.append(result)
        bio_ids.append(str(bio_id))

import itertools
unique_identifiers = list(set(list(itertools.chain.from_iterable(matches))))

print('Unique Identifiers for all samples: ',len(unique_identifiers))

# After this point, I manually joined the results of the above fuzzy match in a spreadsheet. 
# This sheet is imported below. 
joined = pd.read_excel('connecting olink and biorepository data.xlsx')
joined = joined[['raw_data column', 'HorLabBio information']].dropna()

# getting the subset of biorepository info that matches the olink samples that we have
biorepository_item_list_clean = []
for item in list(joined['HorLabBio information']): 
    if isinstance(item, int): 
        biorepository_item_list_clean.append(item)
        continue
    else: 
        biorepository_item_list_clean.append(item.lstrip())
joined['cleaned'] = biorepository_item_list_clean

biorepository_subset = biorepository_info[(biorepository_info['Patient'].isin(biorepository_item_list_clean)) | (biorepository_info['Unnamed: 1'].isin(biorepository_item_list_clean))]
# we have three dataframes biorepository subset | joined | olink data. 
# Step 1: merge biorepository database and the JOINER dataframe
step1 = pd.merge(biorepository_subset, joined, left_on='Patient', right_on='cleaned')
step1_andahalf = pd.merge(biorepository_subset, joined, left_on='Unnamed: 1', right_on='cleaned')

#################
# CREATING STEP 1
#################

step1 = pd.concat([step1_andahalf, step1]).drop_duplicates()
# CREATING UNIQUE PATIENT IDENTIFIERS
patient_list = []
for patient in step1.Patient:
    patient_list.append(str(patient)[0:7])

step1['mrn'] = patient_list

step1['raw_data column'] = step1['raw_data column'].astype(str)
urine = step1[step1['raw_data column'].isin(urine_list)]
urine['sample_type'] = 'urine'
print('Urine dataframe finalized from raw values and biorepository information')

trimmed = urine[[
    'raw_data column', 
    'Response',
    'Treatment ',
    'BCG timepoint',
    'mrn',
    'sample_type'
]]

raw_data = pd.read_excel('Olink data.xlsx', sheet_name='post bt-lot norm')
raw_data = raw_data.set_index('Unnamed: 0').T
raw_data = raw_data.T

urine_only_stacked = raw_data[urine_list].stack().reset_index().rename({
    'Unnamed: 0': 'Cytokine',
    'level_1': 'raw_data column', 
    0: 'OLINK Level'
},
    axis=1
)
urine_only_merged = pd.merge(urine_only_stacked, trimmed, on = 'raw_data column')

custom_dict = {
    '1st dose': 0, 
    '3rd dose': 1, 
    '6th dose': 2, 
    '1st cystoscopy': 3,
    '2nd cystoscopy': 4,
    '3rd cystoscopy': 5,
    '4th cystoscopy': 6,
    '5th cystoscopy': 7,
    '6th cystoscopy': 8,
    '7th cystoscopy': 9,
    '9th Cystoscopy': 10,
} 

urine_only_merged = urine_only_merged.sort_values(by=['BCG timepoint'], key=lambda x: x.map(custom_dict))

urine_only_merged.drop('Treatment ', axis=1, inplace=True)
urine_only_merged = urine_only_merged[['raw_data column','Response', 'BCG timepoint', 'Cytokine', 'OLINK Level']].drop_duplicates()
test = urine_only_merged.pivot(index=['raw_data column','Response', 'BCG timepoint'], columns='Cytokine')

print('Number of identified urine samples prior to any transformations: ',len(urine_list) )
print('Number of urine samples in final dataframe: ', len(set(test.reset_index()['raw_data column'])))

################
# For URINE only
################

heatmap = test.reset_index()
heatmap = heatmap[heatmap['BCG timepoint'].isin(list(custom_dict.keys()))].sort_values(by=['BCG timepoint'], key=lambda x: x.map(custom_dict))

print('Value counts per timepoint')
print(heatmap['BCG timepoint'].value_counts())
print('Total value counts ', heatmap['BCG timepoint'].value_counts().sum())

heatmap.replace({
    'before 1st dose visit': 'Pre-BCG',
    '1st dose': 'Pre-BCG',
    '2nd dose': '2nd Dose', 
    '3rd dose': '3rd Dose',
    '6th dose': '6th Dose', 
    '1st cystoscopy': '1st Cysto',
    '2nd cystoscopy': '2nd Cysto',
    '3rd cystoscopy': '3rd Cysto',
    '4th cystoscopy': '4th Cysto',
    '5th cystoscopy': '5th Cysto',
    '6th cystoscopy': '6th Cysto',
    '7th cystoscopy': '7th Cysto',
    '9th Cystoscopy': '9th Cysto',
}, inplace=True )

In [ ]:
#####################
##### Data Prep #####
#####################

patient_plots = pd.merge(urine_only_stacked, trimmed, on = 'raw_data column')
custom_dict = {
    'before 1st dose visit':0,
    '1st dose': 1, 
    '3rd dose': 2, 
    '6th dose': 3, 
    '1st cystoscopy': 5,
    '2nd cystoscopy': 6,
    '3rd cystoscopy': 7,
    '4th cystoscopy': 8,
    '5th cystoscopy': 9,
    '6th cystoscopy': 10,
    '7th cystoscopy': 11,
    '9th Cystoscopy': 12,
} 

patient_plots = patient_plots.sort_values(by=['BCG timepoint'], key=lambda x: x.map(custom_dict))

patient_plots.replace({
    '1st dose': 'Pre-BCG',
    '3rd dose': '3rd Dose', 
    '6th dose': '6th Dose', 
    '1st cystoscopy': '1st Cysto',
    '2nd cystoscopy': '2nd Cysto',
    '3rd cystoscopy': '3rd Cysto',
    '4th cystoscopy': '4th Cysto',
    '5th cystoscopy': '5th Cysto',
    '6th cystoscopy': '6th Cysto',
    '7th cystoscopy': '7th Cysto',
    '9th Cystoscopy': '9th Cysto',
}, inplace=True )

patient_plots = patient_plots[patient_plots['BCG timepoint'].isin([
'Pre-BCG',
'3rd Dose', 
'6th Dose', 
'1st Cysto',
])]


###################################
##### MAKING THE HEATMAPS #####
###################################

heatmap = heatmap[heatmap['BCG timepoint'].isin(['Pre-BCG', '3rd Dose', '6th Dose', '1st Cysto'])]
heatmap_data = heatmap[['OLINK Level', 'BCG timepoint']].set_index('BCG timepoint')['OLINK Level'].T

heatmap_data.replace({
    '1st dose': 'Pre-BCG',
    '3rd dose': '3rd Dose', 
    '6th dose': '6th Dose', 
    '1st cystoscopy': '1st Cysto',
    '2nd cystoscopy': '2nd Cysto',
    '3rd cystoscopy': '3rd Cysto',
    '4th cystoscopy': '4th Cysto',
    '5th cystoscopy': '5th Cysto',
    '6th cystoscopy': '6th Cysto',
    '7th cystoscopy': '7th Cysto',
    '9th Cystoscopy': '9th Cysto',
}, inplace=True )

heatmap_data['prebcg_mean'] = heatmap_data['Pre-BCG'].mean(axis=1)
heatmap_data['mean'] = heatmap_data.mean(axis=1)
heatmap_data = heatmap_data.sort_values('prebcg_mean')
heatmap_data.loc['mean_bycol'] = heatmap_data.mean()

temp = []
for item in ['Pre-BCG','3rd Dose', '6th Dose', '1st Cysto',]:
    heatmap_data[item] = heatmap_data[item].T.sort_values('mean_bycol', ascending=True).T

# Adding Healthy Control
hc_urine = pd.read_excel('olink_plt5_normed.xlsx')
hc_urine
healthy_urine_controls = ['Unnamed: 0','U_HD1_Amir','U_HD2_Amir','U_HD3_Amir','U_HD4_Amir','U_HD5_Amir','U_HD6_Amir','U_HD7_Amir','U_HD9_Amir','U_HD10_Amir','U_HD11_Amir','U_HD12_Amir',]
hc_urine = hc_urine[healthy_urine_controls]
hc_urine.rename({'Unnamed: 0': 'Cytokine'}, axis=1, inplace=True)
hc_urine.set_index('Cytokine', inplace=True)

# Adding the labels for HCs 
i = 0
newcolnames = []
while i < len(hc_urine.columns):
    newcolnames.append('Healthy Control')
    i+=1
hc_urine.columns = newcolnames
hc_urine

hc_bcg_heatmap = pd.concat([hc_urine, heatmap_data], axis=1)
hc_bcg_heatmap.to_csv('dr to test.csv')

set(hc_bcg_heatmap.columns)
df_list = []
df_list.append(hc_bcg_heatmap['Healthy Control'])
for timepoint in ['Pre-BCG','1st Cysto', '3rd Dose','6th Dose',]:
    df_list.append(hc_bcg_heatmap[timepoint].sort_values(axis=1, by='mean_bycol'))

hc_bcg_heatmap_sorted = pd.concat(df_list, axis=1).drop('mean_bycol')

bcg_bar = patient_plots[['Cytokine', 'OLINK Level','BCG timepoint']]
hc_bar = hc_urine.stack().reset_index().rename({'level_1': 'BCG timepoint', 0:'OLINK Level'}, axis=1)
combined_barchart = pd.concat([hc_bar, bcg_bar])


In [ ]:
# ###################################
# ##### Statistical Significance ####
# ###################################
stat_cytokines = [] 
stat_df_list = []
for cytokine in set(combined_barchart['Cytokine']):
    for dose in ['3rd Dose', '6th Dose', '1st Cysto']: 
        control = combined_barchart[(combined_barchart['Cytokine'] == cytokine) & (combined_barchart['BCG timepoint'] == 'Pre-BCG')]['OLINK Level']
        dose_6 = combined_barchart[(combined_barchart['Cytokine'] == cytokine) & (combined_barchart['BCG timepoint'] == '6th Dose')]['OLINK Level']
        comparator = combined_barchart[(combined_barchart['Cytokine'] == cytokine) & (combined_barchart['BCG timepoint'] == dose)]['OLINK Level']
        dummy, pval_comparator = scipy.stats.shapiro(comparator)
        dummy, pval_control = scipy.stats.shapiro(control)
        # if pvalue for both is > 0.05 we accept the null that both data are normally distributed
        # use ttest ind
        if pval_control > 0.05 and pval_comparator > 0.05:
            stat, pval = scipy.stats.ttest_ind(control, comparator)
            test = 't-test'
        else: 
            stat, pval = scipy.stats.kruskal(control, comparator)
            test = 'Kruskal'
        if pval < 0.05 and comparator.mean() > 0 and control.mean() > 0 and dose_6.mean() - control.mean() > 1:
            stat_cytokines.append(cytokine)
            change = comparator.mean() - control.mean()
            stat_df_list.append([cytokine, dose, change, pval, test])
        
        

###################################
##### Bar Plots #####
###################################

font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 25}

matplotlib.rc('font', **font)

temp = combined_barchart[
        (combined_barchart['Cytokine'].isin(set(stat_cytokines))) & 
        (combined_barchart['BCG timepoint'] != 'Healthy Control')

    ]

# Sorting to cytokines of interest and replacing MCP2 / 4 with their names
temp = temp.replace({'MCP.2':'CCL8', 'MCP.4':'CCL13'})
temp['Cytokine'] = pd.Categorical(temp['Cytokine'], ['IFN.gamma','Flt3L','IL.2RB', 'CCL8','CCL13','CCL19', 'CCL23','CXCL1','CXCL9','CXCL10','CXCL11','TWEAK'])

temp.sort_values("Cytokine")

sns.set(rc={'figure.figsize':(12,14)})
g = sns.barplot(
    y="Cytokine", 
    x="OLINK Level", 
    hue="BCG timepoint", 
    # data=combined_barchart[
    #     (combined_barchart['Cytokine'].isin(set(stat_cytokines))) & 
    #     (combined_barchart['BCG timepoint'] != 'Healthy Control')

    # ], 
    data = temp,
    #palette=['#c9ffd6', '#ff5e5e','#cf0000', '#3b0000', '#ffffff', 'blue', 'orange'],
    palette=['#c9ffd6', '#a7b6ff','#2b50ff', '#000b40', '#000b40', 'blue', 'orange'],
    edgecolor='black',
    linewidth=2.0,
    )
g.axes.xaxis.set_ticks_position("top")
g.set_xlabel('Olink Level')
g.axes.xaxis.set_label_position("top")
g.spines['top'].set_color('black')
g.spines['left'].set_color('black')


# # statistical annotation
lines_xdata = []
lines_ydata = []
for item in g.get_children():
    # print(item == matplotlib.collections.LineCollection)
    if isinstance(item, matplotlib.lines.Line2D):
        lines_xdata.append(item.get_xdata())
        lines_ydata.append(item.get_ydata())

cytokine_location_dict = {}
for counter in range(0,12,1):
    cytokine_location_dict[counter] = []
cytokine_location_dict[counter] = []
for start_point in range(0,12,1):
    for counter in range(start_point,48,12):
        cytokine_location_dict[start_point].append([lines_xdata[counter][1], lines_ydata[counter][0]])

ordered_label_list = []
for item in g.get_yticklabels():
    ordered_label_list.append(item.get_text())


significance = pd.DataFrame(stat_df_list).rename({0:'name', 3:'p'}, axis=1)
significance = significance.replace({'MCP.2':'CCL8', 'MCP.4':'CCL13'})
# correcting for comparisons of cytokines of interest. 
significance['p_adj'] = sm.stats.multipletests(significance['p'], method='fdr_bh')[1]

for key_value_tuple, label in zip(cytokine_location_dict.items(), ordered_label_list):
    values = key_value_tuple[1]
    p = significance[significance['name'] == label]['p_adj'].values[0]
    if p < 0.05: 
        text = '*'
    if p < 0.001: 
        text = '**'
    if p < 0.0001: 
        text = '***'
    x1 = values[0][0] + 0.35
    x2 = values[2][0] + 0.35 + 1
    x3 = values[2][0] + 0.35
    y1 = values[0][1]
    y2 = values[2][1]
    plt.plot([x1, x2, x2, x3], [y1, y1, y2, y2], lw=0.75, c='k')
    plt.text(x2+0.3, y1+0.4, text, ha='center', va='bottom', color='k', size=22)
# Labels, fontsizes, and legend sizes
plt.xlabel('OLINK Urine Level (NPX)',fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylabel('Cytokine',fontsize=20)
legend = plt.legend()
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('black')
plt.setp(g.get_legend().get_texts(), fontsize='18') # for legend text
plt.setp(g.get_legend().get_title(), fontsize='20') # for legend title
plt.savefig('OLINK final figure feb27.png', dpi=200)


# Validation cohort (Denmark)

In [3]:
df_validation = pd.read_csv('OLINK_validation_SiaTrine.csv')
df_plot = df_validation[df_validation['Adjusted_pval'] < 0.05][['Assay','Adjusted_pval', 'Mean_PreBCG', 'Mean_PostBCG']]
df_plot['Assay'] = pd.Categorical(df_plot['Assay'], ['CCL19','CCL23','CXCL9', 'CXCL10','CXCL11', ])

pre = df_plot[['Assay','Mean_PreBCG']].rename({'Mean_PreBCG':'NPX'}, axis=1)
pre['Timepoint'] = 'BCG Naive'

post = df_plot[['Assay','Mean_PostBCG']].rename({'Mean_PostBCG':'NPX'}, axis=1)
post['Timepoint'] = 'BCG Unresponsive'
df_barchart = pd.concat([pre, post])

In [ ]:
sns.set(font_scale=2,rc={'figure.figsize':(12,4)})
g = sns.barplot(
    y="Assay", 
    x="NPX", 
    hue="Timepoint", 
    data = df_barchart,
    alpha=1,
    palette=['#2777B4', '#FF7F0E'],
    edgecolor='black',
    linewidth=2.0,
    )
g.get_legend().remove()

g.axes.xaxis.set_ticks_position("top")
g.set_xlabel('Olink Level')
g.axes.xaxis.set_label_position("top")
g.spines['top'].set_color('black')
g.spines['left'].set_color('black')

# # statistical annotation
lines_ydata = []
lines_xdata = []
for item in g.get_children():
    if isinstance(item, matplotlib.lines.Line2D):
        lines_ydata.append(item.get_ydata())
        

cytokine_location_dict = {}

ordered_label_list = []
for item in g.get_yticklabels():
    ordered_label_list.append(item.get_text())

for item in ordered_label_list:
    cytokine_location_dict[item] = {}
    x1 = df_plot[df_plot['Assay'] == item]['Mean_PreBCG'].values[0]
    x2 = df_plot[df_plot['Assay'] == item]['Mean_PostBCG'].values[0]
    if x1 < 0:
        x1 = 0.1
    
    cytokine_location_dict[item]['x1'] = x1
    cytokine_location_dict[item]['x2'] = x2

for loc, assay in zip(range(0,5,1), ordered_label_list):
    cytokine_location_dict[assay]['y1'] = lines_ydata[loc][0] 
    cytokine_location_dict[assay]['y2'] = lines_ydata[loc+5][0]

for label, value in cytokine_location_dict.items():
    P = df_validation[df_validation == label]['Adjusted_pval'].values[0]
    if p < 0.05: 
        text = '*'
    if p < 0.001: 
        text = '**'
    if p < 0.0001: 
        text = '***'
    x1 = value['x1'] + 0.1
    x2 = value['x2'] + 0.1 + 1
    x3 = value['x2'] + 0.1
    y1 = value['y1']
    y2 = value['y2']
    plt.plot([x1, x2, x2, x3], [y1, y1, y2, y2], lw=0.75, c='k')
    plt.text(x2+0.1, y1+0.6, text, ha='center', va='bottom', color='k', size=22)
    # plt.suptitle('Validation OLINK Urine Cohort: BCG Naive, Recurrence',y=1.05, fontsize=22)
    plt.xlabel('')
    plt.ylabel('')
    plt.savefig('OLINK validation.png', dpi=200)

